# Search

In [61]:
import pandas as pd
import time
from matplotlib import pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [62]:
#Load Tweets
tweets = pd.read_pickle('selected_relevent_tweet.pkl')
tweets = pd.DataFrame(tweets)

In [63]:
#Load Firmlist
firmlist = pd.read_pickle('firmlist.pkl')
# firmlist.drop_duplicates(subset = 'cusp_firm_name', inplace = True)
# firmlist.reset_index(drop = True, inplace = True)



In [64]:
#Brake into all firms that are more and less than 3 chareters long
#If three chareters or less, look for token matches
#If 4 charteres or more look for compelte string matches
longfirms = firmlist[firmlist.length >= 4]
shortfirms = firmlist[firmlist.length <= 3]
longsearch = list(longfirms.search)
shortsearch = list(shortfirms.search)

shortfirms_strip = ['ear', 'ace', 'seg', 'les']
longfirms_strip = ['money', 'chris', 'none', 'tcar', 'same', 'test', "don'tknow", 
                   'peter', 'various', 'unknown', 'charged', 'brooklyn']

for word in shortfirms_strip:
    shortsearch.remove(word)
for word in longfirms_strip:
    longsearch.remove(word)

In [65]:
#Create dataframe with all tweets in a single column along with its category
all_tweets_matter = []
all_cats_matter = []
for cat in tweets:
    for tweet in tweets[cat]:
        if type(tweet) == str:
            all_tweets_matter.append(tweet)
            all_cats_matter.append(cat)

test_tweets = pd.DataFrame(all_tweets_matter, columns= ['tweet'])
test_tweets['cat'] = all_cats_matter


In [71]:
#get rid of twitter handles, whitespace, retweet=RT, and make all lowercase
#replace these cleaned tweets with the old tweets
#drop duplicates
strippedTweets = []
for tweet in test_tweets.tweet:
        #tempTweet = " ".join(filter(lambda x:x[0]!='@', tweet.split()))
        tempTweet = tweet.replace(' ', "")
        tempTweet = tempTweet.replace('RT', "")
        tempTweet = tempTweet.lower()
        strippedTweets.append(tempTweet)
test_tweets['striped_tweet'] = strippedTweets
test_tweets.drop_duplicates(subset = 'striped_tweet', inplace = True)
test_tweets.reset_index(drop = True, inplace = True)

In [74]:
#Create tokens out of tweets
tokenized = []
for tweet in test_tweets.tweet:
    tokenized.append(tweet.split())
test_tweets['tokens'] = tokenized

In [80]:
#find tweets with firm names within the string
jackpot_tweet = []
jackpot_matches = []
jackpot_cat = []
for j in range(len(test_tweets.tokens)):
    if len([i for i in shortsearch if i in test_tweets.tokens[j]]) > 0:
        tempmatches = [i for i in shortsearch if i in test_tweets.tokens[j]]
        jackpot_matches.append(set(tempmatches))
        jackpot_tweet.append(tweets.tweet[j])
        jackpot_cat.append(tweets.cat[j])

for tweet in test_tweets.striped_tweet:
    if any(x in tweet for x in longsearch):
        tempmatches = {x for x in longsearch if x in tweet}
        jackpot_matches.append(tempmatches)
        jackpot_tweet.append(test_tweets.loc[tweets['striped_tweet'] == tweet].tweet.iloc[0])                      

jackpot= pd.DataFrame(jackpot_tweet, columns = ['tweet'])
jackpot['matches'] = jackpot_matches

AttributeError: 'DataFrame' object has no attribute 'tweet'

In [84]:
#All in one function
def tweetSearch(list_of_tweets):
        
    #Load Firmlist and brake into all firms that are more and less than 3 chareters long
    #If three chareters or less, look for token matches
    #If 4 charteres or more look for compelte string matches
    #Remove manual words
    firmlist = pd.read_pickle('firmlist.pkl')
    
    longfirms = firmlist[firmlist.length >= 4]
    shortfirms = firmlist[firmlist.length <= 3]
    longsearch = list(longfirms.search)
    shortsearch = list(shortfirms.search)

    shortfirms_strip = ['ear', 'ace', 'seg', 'les']
    longfirms_strip = ['money', 'chris', 'none', 'tcar', 'same', 'test', "don'tknow", 
                       'peter', 'various', 'unknown', 'charged', 'brooklyn']

    for word in shortfirms_strip:
        shortsearch.remove(word)
    for word in longfirms_strip:
        longsearch.remove(word)
    
    
    
    #get rid of twitter handles, whitespace, retweet=RT, and make all lowercase
    #replace these cleaned tweets with the old tweets
    #drop duplicates
    tweets = pd.DataFrame(list_of_tweets, columns = ['tweet'])
    strippedTweets = []   
    for tweet in tweets.tweet:
            #tempTweet = " ".join(filter(lambda x:x[0]!='@', tweet.split()))
            tempTweet = tweet.replace(' ', "")
            tempTweet = tempTweet.replace('RT', "")
            tempTweet = tempTweet.lower()
            strippedTweets.append(tempTweet)
    tweets['striped_tweet'] = strippedTweets
    tweets.drop_duplicates(subset = 'striped_tweet', inplace = True)
    tweets.reset_index(drop = True, inplace = True)
    
    #Create tokens out of tweets
    tokenized = []
    for tweet in tweets.tweet:
        tokenized.append(tweet.split())
    tweets['tokens'] = tokenized
    
    #find tweets with firm names within the string
    jackpot_tweet = []
    jackpot_matches = []
    jackpot_cat = []
    for j in range(len(tweets.tokens)):
        if len([i for i in shortsearch if i in tweets.tokens[j]]) > 0:
            tempmatches = [i for i in shortsearch if i in tweets.tokens[j]]
            jackpot_matches.append(set(tempmatches))
            jackpot_tweet.append(tweets.tweet[j])

    for tweet in tweets.striped_tweet:
        if any(x in tweet for x in longsearch):
            tempmatches = {x for x in longsearch if x in tweet}
            jackpot_matches.append(tempmatches)
            jackpot_tweet.append(tweets.loc[tweets['striped_tweet'] == tweet].tweet.iloc[0])
            
    jackpot= pd.DataFrame(jackpot_tweet, columns = ['tweet'])
    jackpot['matches'] = jackpot_matches
    
    now_its_a_list = []
    for i in jackpot.matches:
        if len (list(i)) > 0:
            now_its_a_list.append(list(i))
        else:
            now_its_a_list.append(['no_company_listed'])
    jackpot['matches'] = now_its_a_list
    
    return(jackpot, list(jackpot.tweet), list(jackpot.matches))

In [85]:
# Test to see how long the function takes
start = time.time()
x = tweetSearch(list(test_tweets.tweet))
end = time.time()
(end - start) / 60.0

0.05234489838282267

In [ ]:
#just one tweet
# def tweetSearch_for_one_tweet(tweets):
    
#     #Load Firmlist and brake into all firms that are more and less than 3 chareters long
#     #If three chareters or less, look for token matches
#     #If 4 charteres or more look for compelte string matches
#     #Remove manual words
#     firmlist = pd.read_pickle('firmlist.pkl')
    
#     longfirms = firmlist[firmlist.length >= 4]
#     shortfirms = firmlist[firmlist.length <= 3]
#     longsearch = list(longfirms.search)
#     shortsearch = list(shortfirms.search)

#     shortfirms_strip = ['ear', 'ace', 'seg', 'les']
#     longfirms_strip = ['money', 'chris', 'none', 'tcar', 'same', 'test', "don'tknow", 
#                        'peter', 'various', 'unknown', 'charged', 'brooklyn']

#     for word in shortfirms_strip:
#         shortsearch.remove(word)
#     for word in longfirms_strip:
#         longsearch.remove(word)
        
#     #get rid of twitter handles, whitespace, retweet=RT, and make all lowercase
#     #replace these cleaned tweets with the old tweets
#     #drop duplicates
    
    
    
#     stripped_tweet = " ".join(filter(lambda x:x[0]!='@', tweet.split()))
#     stripped_tweet = stripped_tweet.replace(' ', "")
#     stripped_tweet = stripped_tweet.replace('RT', "")
#     stripped_tweet = stripped_tweet.lower()
    
#     #Create tokens out of tweets
#     tokenized_tweet = tweet.split()
    
#     #find tweets with firm names within the string
#     if len([i for i in shortsearch if i in tokenized_tweet]) > 0:
#         matches = set([i for i in shortsearch if i in tokenized_tweet])
#     #long search in stripped tweets
#     elif any(x in tweet for x in longsearch):
#         matches = {x for x in longsearch if x in tweet}
        
#     else:
#         matches = ['no_company_mentioned']

#     return(tweet, matches)